# Dataset: **

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

In [ ]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src.csv', encoding = 'utf-8', header =0)

In [ ]:
df.isnull().sum()

In [ ]:
src_tw_df.isnull().sum()

In [ ]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)

## Emotion Analysis

### T5 Based
<https://huggingface.co/mrm8488/t5-base-finetuned-emotion?text=I+wish+you+were+here+but+it+is+impossible>

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion",use_fast=False)
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")



def get_emotion_t5(text):
    input_ids = t5_tokenizer.encode(text , return_tensors='pt')
    output = t5_model.generate(input_ids=input_ids,max_length=2)
  
    dec = [t5_tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

In [ ]:
emotion_t5 = df.cleaned_reply_tw.astype(str).apply(get_emotion_t5)

In [ ]:
df['reply_emo_t5'] = emotion_t5

In [ ]:
import re
df['reply_emo_t5'] = df['reply_emo_t5'].apply(lambda x : re.sub('<pad> ',"",x))

In [ ]:
src_emo_t5 = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_t5)
src_tw_df['src_emo_t5'] = src_emo_t5

In [ ]:
src_tw_df['src_emo_t5']  = src_tw_df.src_emo_t5.apply(lambda x : re.sub('<pad> ',"",x))

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [ ]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [ ]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [ ]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [ ]:
df['reply_emo_ro'] = emotion_ro

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df-2.csv',index=False)

In [ ]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [ ]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [ ]:
print("source tweet:", df.src_tweet[1118],'\n')
print("original reply tw: ", df.reply_tweet[1118])
print("cleaned reply tw: ", df.cleaned_reply_tw[1118])

print('roBERT based: ',df.reply_emo_ro[1118])
print('t5 based: ',df.reply_emo_t5[1118])
print('text2emotion: ', df.reply_emo_te[1118])